来源：[Fuzzing: Breaking Things with Random Inputs](https://www.fuzzingbook.org/html/Fuzzer.html)

## 模糊测试的简单事例

In [ ]:
# 生成一个指定范围内，随机长度，随机字母的字符串
import random
def fuzzer(min_length=5,max_length=15,char_start=ord('a'),char_range=26):
    str_length = random.randrange(min_length,max_length+1)
    out = ""
    for i in range(str_length):
        out += chr(random.randrange(char_start,char_start+char_range))
    return out

In [ ]:
# 将生成的字符串写入文件
import os
import tempfile
basename = "input.txt"
tmp_dir = tempfile.mkdtemp()
FILE = os.path.join(tmp_dir,basename)

data = fuzzer()
with open(FILE,"w") as f:
    f.write(data)
print(open(FILE).read())

In [ ]:
# 调用外部程序
import subprocess
program = "bc"
with open(FILE, "w") as f:
    f.write("2 + 2\n")
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7
# print(result)
print(result.stdout)
# print(result.returncode)
# print(result.stderr)

In [ ]:
# 测试bc程序
trials = 50
program = "bc"
results = []
for i in range(trials):
    data = fuzzer(min_length=2,char_start=ord('0'))
    data += '\n'
    with open(FILE,"w") as f:
        f.write(data)
    result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7
    results.append((data,result))

In [ ]:
# 测试bc程序的结果分析，没有返回值为非0存在，即没有崩溃
# 如果有返回代码非0，断言
sum_suc = 0
for i in range(trials):
    (data,result) = results[i]
    assert result.returncode == 0
    if result.stderr == "":
        sum_suc += 1
        print(f"suc：{data}-->{result.stdout}")
    else:
        print(f"fail：{data}-->{result.stderr}")
print(sum_suc)

In [ ]:
# 创建可能会内存访问越界的程序
with open("program.c", "w") as f:
    f.write("""
#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    """)

In [ ]:
# 使用clang编译；AddressSanitizer is a fast memory error detector.
# gcc应该也有这个功能吧
!gcc -fsanitize=address -g -o program program.c
# !clang -fsanitize=address -g -o program program.c

In [ ]:
# 正常，输出42
!./program 99; echo $?

In [ ]:
# 错误，SUMMARY: AddressSanitizer: heap-buffer-overflow
!./program 100; echo $?

In [ ]:
# 删除程序
!rm -fr program program.*

## 构建模糊测试架构 -- Runner class

In [ ]:
# 实例变量用于每个实例的唯一数据，而类变量用于类的所有实例共享的属性和方法
# 如果同样的属性名称同时出现在实例和类中，则属性查找会优先选择实例

# base runner class ： essentially provides a method run(input)，run() returns a pair (result, outcome)
# result是返回值的详细信息，outcome是三种分类之一
# 这个类是基类，下面通过继承覆盖，产生不同的子类
class Runner(object):
    # Test outcomes
    PASS = "PASS"
    FAIL = "FAIL"
    UNRESOLVED = "UNRESOLVED"
    def __init__(self):
        pass
    def run(self,inp):
        return(inp, self.UNRESOLVED)

In [ ]:
# 继承Runner类。打印输入
class PrintRunner(Runner):
    def run(self,inp):
        print(inp)
        return(inp, self.UNRESOLVED)

In [ ]:
# 继承Runner类
# 把输入发送给程序；程序在创建对象的时候制定
class ProgramRunner(Runner):
    def __init__(self,program):
        self.program = program
    def run_process(self,inp=""):
        return subprocess.run(self.program,
                                input=inp,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)
                                # text=True) 我的是3.6版本，还没有text
    def run(self,inp=""):
        result = self.run_process(inp)
        if result.returncode == 0:
            outcome = self.PASS
        elif result.outcome < 0:
            outcome = self.FAIL
        else:
            outcome = self.UNRESOLVED
        return (result,outcome)

In [ ]:
# 继承ProgramRunner类
# 如果输入是二进制形式
class BinaryProgramRunner(ProgramRunner):
    def run_process(self,inp=""):
        return subprocess.run(self.program,
                            input=inp.encode(),
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

In [ ]:
# 测试下ProgramRunner
cat = ProgramRunner(program="cat")
(result,outcome) = cat.run("I am cat")
print(result)

## 构建模糊测试架构 -- Fuzzer class

Fuzzer 类主要是创建随机输入，用以喂给run()方法；

In [ ]:
# Fuzzer基类：生成随机输入，并使用run方法运行
class Fuzzer(object):
    def __init__(self):
        pass
    def fuzz(self):
        return ""
    def run(self,runner=Runner()):
        return runner.run(self.fuzz())
    def runs(self,runner=PrintRunner(),trials=10):
        outcomes = []
        for i in range(trials):
            # outcomes.append(runner.run(self.fuzz()))
            outcomes.append(self.run(runner))
        return outcomes

In [ ]:
class RandomFuzzer(Fuzzer):
    def __init__(self, min_length=10, max_length=100,char_start=32, char_range=32):
        self.min_length = min_length
        self.max_length = max_length
        self.char_start = char_start
        self.char_range = char_range
    def fuzz(self):
        str_len = random.randrange(self.min_length,self.max_length)
        out = ""
        for i in range(str_len):
            out += chr(random.randrange(self.char_start,self.char_start + self.char_range))
        return out

In [ ]:
# 测试RandomFuzzer
random_fuzzer = RandomFuzzer(min_length=5,max_length=10,char_start=ord('a'),char_range=26)
# random_fuzzer.fuzz() # 可以随机生成字符串，很好
cat_runner = ProgramRunner("cat")
outcomes = random_fuzzer.runs(cat_runner,10)
print(outcomes)

In [ ]:
# 上面的bc测试使用我们的基类
random_fuzzer = RandomFuzzer(min_length=2,max_length=6,char_start=ord('0'),char_range=10)
bc_runner = ProgramRunner("bc")
outcomes = random_fuzzer.runs(bc_runner,10)
print(outcomes)